In [3]:
import tweepy
import pandas as pd

In [4]:
# Credentials

consumer_key = 'frxFyJskSPkPvxiMu8NKUfvX8'

consumer_secret = 'tCtujcAsHIZF8TSzDxKCBJbfM75v8vNEzClmuA48nSqpnC0PH0'

access_token = '557446265-KT2R47ApGYDLiLMDjbYTQONxhctzAtdhbZRm39SE'

access_token_secret = 'nYaLC8D3KzHPcKCRNIoNclAEiMDZ5mROmCqpZZsyheAxb'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)

In [6]:
screen_name = input('Enter user twitter handle: ')

user = api.get_user(str(screen_name))


print(user)
print("The name is : " + user.name)
print("The screen_name is : " + user.screen_name)
print("The location is : " + str(user.location))
print("The profile_location is : " + str(user.profile_location))
print("The description is : " + user.description)

Enter user twitter handle: saifuddintasnim
User(_api=<tweepy.api.API object at 0x000002023268B1C0>, _json={'id': 557446265, 'id_str': '557446265', 'name': 'Saifuddin Tasnim', 'screen_name': 'saifuddintasnim', 'location': 'Singapore', 'profile_location': None, 'description': "| Feminist |`94 | SG | I always thought my life is a tragedy but now i realize it's a comedy - Arthur Fleck", 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 82, 'friends_count': 134, 'listed_count': 0, 'created_at': 'Thu Apr 19 05:22:10 +0000 2012', 'favourites_count': 527, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 15119, 'lang': None, 'status': {'created_at': 'Mon Sep 27 17:04:23 +0000 2021', 'id': 1442535601421443072, 'id_str': '1442535601421443072', 'text': 'RT @Acap_aan: Haihhh tetiba mata masuk habuk 🥺🥺🥺 https://t.co/CMCWpSp7QK', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions':